In [1]:
import pandas as pd
import json

In [ ]:
#I can use pd.read_csv("passwords1.txt", header=None, delim_whitespace=True) becvause it will skip some lines.

In [19]:
i = 0
with open("passwords1.txt") as infile:
    for line in infile:
        i = i+1
print('Number of passwords in passwords1.txt', i)

Number of passwords in passwords1.txt 100000000


In [20]:
i = 0
with open("passwords2.txt") as infile:
    for line in infile:
        i = i+1
print('Number of passwords in passwords2.txt', i)

Number of passwords in passwords2.txt 39000000


In [9]:
ord('a')

97

In [8]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


We choose m(dimension of the future bit array) in respect of a 7% false positive rate that we will pose in the future bloom filter, so the module of the hash fucntions will the the first greater prime of 553490135.

In [2]:
import sympy as s
m = s.nextprime(553490135)
m

553490137

In [3]:
import random
random.randrange(m)

313948731

We will create 4 hash functions, that we will later describe, the first thing we that I need 4 vectors with 20 components, all randomly generated in between 0 and m.

In [4]:
def randomizr(m): 
    vec = []
    for i in range(20):
        vec.append(random.randrange(m))
    return vec


In [51]:
vector1 = randomizr(m)
vector2 = randomizr(m)
vector3 = randomizr(m)
vector4 = randomizr(m)
print(vector1,'\n',vector2, '\n', vector3, '\n', vector4)
print(vector1 == vector2 or vector1==vector3 or vector1==vector4 or vector2 == vector3 or vector3==vector4)

[548373287, 222232648, 393628080, 220566875, 315011190, 457014546, 401445075, 165870502, 544231662, 431020630, 189088991, 382246549, 207652822, 296995918, 282393338, 361281726, 379519948, 289192586, 422903918, 179727888] 
 [510338572, 428859708, 127662473, 411969966, 268400108, 235368761, 316744780, 506406582, 545941917, 173198424, 519259740, 487278310, 165823038, 452155477, 51364823, 418169255, 277375778, 416769453, 246767791, 492198197] 
 [372907205, 539742227, 324783969, 297544716, 334245771, 88148016, 531791743, 323977773, 386017264, 22958949, 341277333, 215029602, 313390962, 343400267, 91334501, 510518004, 437634524, 319955216, 323354422, 493771037] 
 [429698059, 384014737, 257859287, 93358385, 410243180, 386311450, 514566367, 450082373, 480763392, 117581539, 403105809, 121520178, 313661122, 13845469, 325510496, 256279553, 371541715, 133092061, 56891456, 90041203]
False


Now that the vectors are all different, to avoid changing them every time we run the file, we save the vectors with the values printed above. We did that to make explicit the uniform distribution to chooes those components.

In [5]:
vector1 = [548373287, 222232648, 393628080, 220566875, 315011190, 457014546, 401445075, 165870502, 544231662, 431020630, 189088991, 382246549, 207652822, 296995918, 282393338, 361281726, 379519948, 289192586, 422903918, 179727888]
vector2 = [510338572, 428859708, 127662473, 411969966, 268400108, 235368761, 316744780, 506406582, 545941917, 173198424, 519259740, 487278310, 165823038, 452155477, 51364823, 418169255, 277375778, 416769453, 246767791, 492198197]
vector3 = [372907205, 539742227, 324783969, 297544716, 334245771, 88148016, 531791743, 323977773, 386017264, 22958949, 341277333, 215029602, 313390962, 343400267, 91334501, 510518004, 437634524, 319955216, 323354422, 493771037]
vector4 = [429698059, 384014737, 257859287, 93358385, 410243180, 386311450, 514566367, 450082373, 480763392, 117581539, 403105809, 121520178, 313661122, 13845469, 325510496, 256279553, 371541715, 133092061, 56891456, 90041203]
#done!

In [6]:
import numpy as np
def simple_dot(a, b):
    dsum = 0.
    for ((idx,), val) in np.ndenumerate(a):
        dsum += int(val) * int(b[idx])
    return int(dsum)

In [54]:
simple_dot([1,2], [4,5])

14

Now we define the 4 hash fucntions, that will actually be the same but with different coefficients given by the vectors we just defined.

In [7]:
def Hash1(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector1)%m)


In [8]:
def Hash2(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector2)%m)

In [9]:
def Hash3(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector3)%m)

In [10]:
def Hash4(x): 
    vec = []
    for i in x:
        vec.append(ord(i))
    return (simple_dot(vec,vector4)%m)

Now that I have 4 Hash functions I need to build the bloom filter!

In [11]:
import time
def BloomFilter(passwords1, passwords2):
    start = time.time()
    filtr = [0]*553490137 #here m. the module
    probably = 0
    #first i put in my filter all the password from passwords1, signing with a 1 the components where there is something
    with open(passwords1) as infile:
        for line in infile:
            item = line.replace('\n','')
            if filtr[Hash1(item)] == 0:
                filtr[Hash1(item)] = 1
            if filtr[Hash2(item)] == 0:
                filtr[Hash2(item)] = 1
            if filtr[Hash3(item)] == 0:
                filtr[Hash3(item)] = 1
            if filtr[Hash4(item)] == 0:
                filtr[Hash4(item)] = 1
    #now that every password from df passwords1 is inside, we have to put the new ones and count homw many are probably duplicates
    with open(passwords2) as infile:
        for line in infile:
            item = line.replace('\n','')
            if filtr[Hash1(item)] == 1 and  filtr[Hash2(item)] == 1 and filtr[Hash2(item)] == 1 and filtr[Hash2(item)] == 1 :
                probably = probably + 1
    end = time.time()
    #for the probability of false positive I build the model from the passwords1 size and i false positive rate i wanted, so it's 7%
    print('Number of hash function used: ', 4)
    print('Number of duplicates detected: ', probably)
    print('Probability of false positives: ', 0.07)
    print('Execution time: ', end-start)
    return 
        

In [ ]:
BloomFilter('passwords1.txt', 'passwords2.txt')

Now we will modify a little the bloom filter to asnwer the bonus question about the exact number of false positives.

In [ ]:
import time
def BloomFilter2(passwords1, passwords2):
    start = time.time()
    filtr = [0]*553490137 #here m. the module
    false_pos = 0
    #first i put in my filter all the password from passwords1, signing with a 1 the components where there is something
    dic_hash1 = {}
    with open(passwords1) as infile:
        for line in infile:
            item = line.replace('\n','')
            A = Hash1(item)
            if filtr[Hash1(item)] == 0:
                filtr[Hash1(item)] = 1
                #we create a dictionary where i will access to verify if the password that has all 1 in the filter is really in the file.
                dic_hash1[A] = [item]
            else:
                dic_hash1[A].append(item)
            if filtr[Hash2(item)] == 0:
                filtr[Hash2(item)] = 1
            if filtr[Hash3(item)] == 0:
                filtr[Hash3(item)] = 1
            if filtr[Hash4(item)] == 0:
                filtr[Hash4(item)] = 1
    #now that every password from df passwords1 is inside, we have to put the new ones and count homw many are probably duplicates
    with open(passwords2) as infile:
        for line in infile:
            item = line.replace('\n','')
            A = Hash1(item)
            if filtr[A] == 1 and  filtr[Hash2(item)] == 1 and filtr[Hash2(item)] == 1 and filtr[Hash2(item)] == 1 :
                if item not in filtr[A]:
                    false_pos = false_pos + 1
    end = time.time()
    #for the probability of false positive I build the model from the passwords1 size and i false positive rate i wanted, so it's 7%
    print('Number of hash function used: ', 4)
    print('Number of false positives: ', false_pos)
    print('Theoretical probability of false positives: ', 0.07)
    print('Actual false positives probability', false_pos/float(39000000))
    print('Execution time: ', end-start)
    return 

In [ ]:
BloomFilter2('passwords1.txt', 'passwords2.txt')